In [1]:
%%capture --no-stderr
%pip install --quiet -U langchain_google_genai langchain_core langgraph

In [2]:
from pprint import  pprint
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage, AnyMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph,START,END
from typing_extensions import TypedDict
from typing import Annotated



## Messages

In [3]:
messages = [AIMessage(content="Hi there! How can I assist today?", name="Model")]

messages.append(HumanMessage(content="Count the words of 'I am Chanaka'", name="Chanaka"))
messages.append(AIMessage(content="The word count is 3", name="Model"))
messages.append(HumanMessage(content="Can you also reverse the text 'LangGraph is awesome'?", name="Chanaka"))

for m in messages:
    m.pretty_print()


================================== Ai Message ==================================
Name: Model

Hi there! How can I assist today?
================================ Human Message =================================
Name: Chanaka

Count the words of 'I am Chanaka'
================================== Ai Message ==================================
Name: Model

The word count is 3
================================ Human Message =================================
Name: Chanaka

Can you also reverse the text 'LangGraph is awesome'?


In [4]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("GOOGLE_API_KEY")

GOOGLE_API_KEY: ··········


In [5]:
llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash')
result = llm.invoke("Hi there")
result.content

'Hello! How can I help you today?'

## Tools

In [6]:
def reverse_text(text: str) -> str:
  """
  Reverse a string
  """
  return text[::-1]

def count_words(text: str) -> str:
  """
  Count number of words in a string
  """
  return len(text.split())

In [7]:
llm_with_tools = llm.bind_tools([reverse_text,count_words])

In [8]:
result = llm_with_tools.invoke("Count the words of 'I am Chanaka'")
result

AIMessage(content='', additional_kwargs={'function_call': {'name': 'count_words', 'arguments': '{"text": "I am Chanaka"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--2534de8d-c4df-45cc-88ee-646e07838ffd-0', tool_calls=[{'name': 'count_words', 'args': {'text': 'I am Chanaka'}, 'id': '56ce380c-b673-4ceb-aaee-0cee94c3b2d9', 'type': 'tool_call'}], usage_metadata={'input_tokens': 85, 'output_tokens': 84, 'total_tokens': 169, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 66}})

In [9]:
result.tool_calls

[{'name': 'count_words',
  'args': {'text': 'I am Chanaka'},
  'id': '56ce380c-b673-4ceb-aaee-0cee94c3b2d9',
  'type': 'tool_call'}]

In [10]:
result = llm_with_tools.invoke("Reverse 'I am Chanaka'")
result

AIMessage(content='', additional_kwargs={'function_call': {'name': 'reverse_text', 'arguments': '{"text": "I am Chanaka"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--80d1f530-b82b-4342-969d-53bc0c460bb9-0', tool_calls=[{'name': 'reverse_text', 'args': {'text': 'I am Chanaka'}, 'id': 'f9e70df3-48aa-44f7-89be-60f1bfd714f4', 'type': 'tool_call'}], usage_metadata={'input_tokens': 82, 'output_tokens': 93, 'total_tokens': 175, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 75}})

In [11]:
result.tool_calls

[{'name': 'reverse_text',
  'args': {'text': 'I am Chanaka'},
  'id': 'f9e70df3-48aa-44f7-89be-60f1bfd714f4',
  'type': 'tool_call'}]

In [12]:
result = llm_with_tools.invoke("Count and reverse: 'I am Chanaka'")
result.tool_calls

[{'name': 'count_words',
  'args': {'text': 'I am Chanaka'},
  'id': 'f98ad523-2476-4887-9ae7-b566017242e6',
  'type': 'tool_call'},
 {'name': 'reverse_text',
  'args': {'text': 'I am Chanaka'},
  'id': '3a2260b3-98c7-492d-a17a-c2b8cc094908',
  'type': 'tool_call'}]

In [13]:
result = llm_with_tools.invoke("What is AI?")
result.tool_calls

[]

## Reducers


In [14]:
class MessagesState(TypedDict):
    messages: list[AnyMessage]

# Node
def tool_calling_llm(state:MessagesState):
  return {"messages":[llm_with_tools.invoke(state['messages'])]}

builder = StateGraph(MessagesState)
builder.add_node("tool_calling_node",tool_calling_llm)
builder.add_edge(START,"tool_calling_node")
builder.add_edge("tool_calling_node",END)

graph = builder.compile()

In [15]:
result =  graph.invoke({"messages":[HumanMessage(content="Count and reverse: 'I am Chanaka'")]})
for m in result['messages']:
    m.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  count_words (a2adace5-a9d0-40a0-9fb3-088a39664a41)
 Call ID: a2adace5-a9d0-40a0-9fb3-088a39664a41
  Args:
    text: I am Chanaka
  reverse_text (38ec092b-91bf-4e37-9547-18dfc082d557)
 Call ID: 38ec092b-91bf-4e37-9547-18dfc082d557
  Args:
    text: I am Chanaka


In [16]:
class MessagesState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

# Node
def tool_calling_llm(state:MessagesState):
  return {"messages":[llm_with_tools.invoke(state['messages'])]}

builder = StateGraph(MessagesState)
builder.add_node("tool_calling_node",tool_calling_llm)
builder.add_edge(START,"tool_calling_node")
builder.add_edge("tool_calling_node",END)

graph = builder.compile()


In [17]:
result =  graph.invoke({"messages":[HumanMessage(content="Count and reverse: 'I am Chanaka'")]})

for m in result['messages']:
    m.pretty_print()

================================ Human Message =================================

Count and reverse: 'I am Chanaka'
================================== Ai Message ==================================
Tool Calls:
  count_words (0dfd8fd9-607e-4032-be9f-aedbf9663fc9)
 Call ID: 0dfd8fd9-607e-4032-be9f-aedbf9663fc9
  Args:
    text: I am Chanaka
  reverse_text (9b0df3e4-9c3c-4ef2-be02-db078a1bbd44)
 Call ID: 9b0df3e4-9c3c-4ef2-be02-db078a1bbd44
  Args:
    text: I am Chanaka
